In [8]:
import os
import xlwings as xw

def set_second_sheet_focus_a1(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing: {filename}")

            app = xw.App(visible=False)
            wb = app.books.open(file_path)

            if len(wb.sheets) >= 2:
                ws = wb.sheets[1]  # second sheet
                ws.activate()
                ws.range('A1').select()
                wb.save()
                print(f"Set A1 on 2nd sheet: {filename}")
            else:
                print(f"Skipped (not enough sheets): {filename}")

            wb.close()
            app.quit()

# Example usage:
set_second_sheet_focus_a1(r'C:\\Users\\KS\\K')


Processing: sample_excel - Copy (2).xlsx
Set A1 on 2nd sheet: sample_excel - Copy (2).xlsx
Processing: sample_excel - Copy (3).xlsx
Set A1 on 2nd sheet: sample_excel - Copy (3).xlsx
Processing: sample_excel - Copy (4).xlsx
Set A1 on 2nd sheet: sample_excel - Copy (4).xlsx
Processing: sample_excel - Copy (5).xlsx
Set A1 on 2nd sheet: sample_excel - Copy (5).xlsx
Processing: sample_excel - Copy (6).xlsx
Set A1 on 2nd sheet: sample_excel - Copy (6).xlsx
Processing: sample_excel - Copy (7).xlsx
Set A1 on 2nd sheet: sample_excel - Copy (7).xlsx
Processing: sample_excel - Copy (8).xlsx
Set A1 on 2nd sheet: sample_excel - Copy (8).xlsx
Processing: sample_excel - Copy.xlsx
Set A1 on 2nd sheet: sample_excel - Copy.xlsx
Processing: sample_excel.xlsx
Set A1 on 2nd sheet: sample_excel.xlsx


In [ ]:
# Create 3 blank rows matching the number of columns
blank_row = [''] * len(df_out.columns)
blank_df = pd.DataFrame([blank_row] * 3)

# Then stack: blanks → year → header → data
df_out_final = pd.concat(
    [blank_df, pd.DataFrame([year_row]), pd.DataFrame([header_row]), df_out],
    ignore_index=True
)

In [5]:
# paste as valeus
import os
import win32com.client as win32

def paste_values_via_vba(folder_path):
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.Visible = False

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing: {filename}")

            wb = excel.Workbooks.Open(file_path)
            for ws in wb.Worksheets:
                ws.Cells.Copy()
                ws.Cells.PasteSpecial(Paste=-4163)  # xlPasteValues
            excel.CutCopyMode = False
            wb.Save()
            wb.Close()

    excel.Quit()
    print("Done!")

paste_values_via_vba(r'C:\\Users\\KS\\K')


Processing: sample_excel - Copy (2).xlsx
Processing: sample_excel - Copy (3).xlsx
Processing: sample_excel - Copy (4).xlsx
Processing: sample_excel - Copy (5).xlsx
Processing: sample_excel - Copy (6).xlsx
Processing: sample_excel - Copy (7).xlsx
Processing: sample_excel - Copy (8).xlsx
Processing: sample_excel - Copy.xlsx
Processing: sample_excel.xlsx
Done!


In [2]:
import os
import xlwings as xw

def paste_values_all_sheets(folder_path):
    for filename in os.listd`ir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing: {filename}")

            app = xw.App(visible=False)
            wb = app.books.open(file_path)

            # Select all sheets using the COM API
            sheet_names = [sheet.name for sheet in wb.sheets]
            wb.api.Worksheets(sheet_names).Select()

            # Loop through each selected sheet
            for sheet in wb.sheets:
                used_range = sheet.used_range
                used_range.api.Copy()
                used_range.api.PasteSpecial(-4163)  # xlPasteValues

            wb.save()
            wb.close()
            app.quit()

    print("Done!")

# Example usage:
paste_values_all_sheets(r'Ok')


Processing: sample1.xlsx
Processing: sample_excel.xlsx
Processing: sample_One.xlsx
Processing: ~$sample1.xlsx


com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', 'Excel cannot open the file ’~$sample1.xlsx’ because the file format or file extension is not valid. Verify that the file has not been corrupted and that the file extension matches the format of the file.', 'xlmain11.chm', 0, -2146827284), None)

In [ ]:
import pandas as pd

# === USER INPUT ===
current_month = input("Enter month (e.g. M06): ")
base_year = 2025
other_year = 2024

input_file = "your_input_file.xlsx"  # <<-- UPDATE to your file name!
xls = pd.ExcelFile(input_file)

# === Identify sheets ===
sheets_to_process = [s for s in xls.sheet_names if 'actual' in s.lower()]
sheets_to_copy = [s for s in xls.sheet_names if s not in sheets_to_process]

output_sheets = {}

# === Process each 'Actual' sheet ===
for sheet in sheets_to_process:
    print(f"Processing: {sheet}")

    # === Detect header ===
    df_raw = pd.read_excel(xls, sheet_name=sheet, header=None)
    header_row_idx = df_raw.apply(
        lambda row: row.astype(str).str.contains('Account', case=False).any()
        and row.astype(str).str.contains('Entity', case=False).any()
        and row.astype(str).str.contains('Level 4', case=False).any(),
        axis=1
    ).idxmax()

    df = pd.read_excel(xls, sheet_name=sheet, header=header_row_idx)

    # === Normalize column names ===
    df.columns = df.columns.str.strip().str.replace(' ', '')
    df.columns = df.columns.str.replace('YTD$', 'YTD12')

    # === Key columns ===
    key_cols = ['Account', 'Entity', 'Level4']
    df.rename(columns={'Level4': 'Level4'}, inplace=True)  # just in case

    # === Master key ===
    all_keys = df[key_cols].drop_duplicates()

    # === Base columns ===
    base_cols = ['YTD12'] + [f"M{str(i).zfill(2)}" for i in range(1, 13)]

    # === Start output ===
    df_out = all_keys.copy()

    # === Fill 2025 block ===
    df_2025 = df[df['Year'] == base_year][key_cols + base_cols].copy()
    df_2025 = df_2025.groupby(key_cols, as_index=False).sum()
    df_out = df_out.merge(df_2025, on=key_cols, how='left').fillna(0)

    # === Fill 2024 YTD12 ===
    df_2024 = df[df['Year'] == other_year][key_cols + ['YTD12']].copy()
    df_2024 = df_2024.groupby(key_cols, as_index=False).sum()
    df_out = df_out.merge(df_2024, on=key_cols, how='left', suffixes=('', f'_{other_year}')).fillna(0)

    # === Final column order ===
    df_out = df_out[key_cols + base_cols + [f'YTD12_{other_year}']]

    # === Reset real columns so there's no shift when saving ===
    df_out.columns = range(len(df_out.columns))

    # === Build final header rows ===
    year_row = (
        [''] * 3 +
        [str(base_year)] * len(base_cols) +
        [str(other_year)]
    )
    header_row = key_cols + base_cols + ['YTD12']

    # === Combine header rows + data ===
    df_out_final = pd.concat(
        [pd.DataFrame([year_row]), pd.DataFrame([header_row]), df_out],
        ignore_index=True
    )

    output_sheets[sheet] = df_out_final

# === Copy other sheets ===
for sheet in sheets_to_copy:
    print(f"Copying unchanged: {sheet}")
    df = pd.read_excel(xls, sheet_name=sheet)
    output_sheets[sheet] = df

# === Write final output ===
output_file = "Total PV.xlsx"
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name, df in output_sheets.items():
        df.to_excel(
            writer,
            index=False,
            header=False if sheet_name in sheets_to_process else True,
            sheet_name=sheet_name
        )

print(f"✅✅✅ DONE! Output saved as: {output_file}")


In [ ]:
from openpyxl import load_workbook

def fix_product_control_bfg(file_path):
    wb = load_workbook(file_path)
    ws = wb["Data"]

    # Assume header is in row 2
    headers = [cell.value.strip() if cell.value else None for cell in ws[2]]

    try:
        bf_idx = headers.index("Business Framework") + 1  # openpyxl is 1-based
        bfg_idx = headers.index("Business Framework Group") + 1
    except ValueError:
        print("Required columns not found.")
        return

    updated_count = 0

    for row in ws.iter_rows(min_row=3, max_row=ws.max_row):
        bf_value = row[bf_idx - 1].value
        if bf_value and bf_value.strip().upper() == "PRODUCT CONTROL":
            bfg_cell = row[bfg_idx - 1]
            if bfg_cell.value != "L3 - Financial Control & Tax":
                bfg_cell.value = "L3 - Financial Control & Tax"
                updated_count += 1

    wb.save(file_path)
    wb.close()

    print(f"✅ Done. Updated {updated_count} rows.")

# Example usage:
fix_product_control_bfg("YourReport.xlsx")


In [ ]:
import win32com.client as win32

def update_product_control(filepath):
    # Start Excel
    excel = win32.gencache.EnsureDispatch("Excel.Application")
    excel.Visible = False  # True for debugging

    wb = excel.Workbooks.Open(filepath)
    ws = wb.Sheets("Data")

    # Find last used row
    last_row = ws.Cells(ws.Rows.Count, 1).End(-4162).Row  # xlUp

    # Find columns
    headers = [ws.Cells(2, col).Value for col in range(1, ws.UsedRange.Columns.Count + 1)]
    bf_col = headers.index("Business Framework") + 1
    bfg_col = headers.index("Business Framework Group") + 1

    # Apply AutoFilter on Business Framework
    ws.Range(ws.Cells(2, bf_col), ws.Cells(last_row, bfg_col)).AutoFilter(Field=bf_col, Criteria1="PRODUCT CONTROL")

    # Get visible rows in filter (skip header)
    visible = ws.Range(ws.Cells(3, bf_col), ws.Cells(last_row, bf_col)).SpecialCells(12)  # 12 = xlCellTypeVisible
    rows = sorted([cell.Row for cell in visible])

    if not rows:
        print("✅ No PRODUCT CONTROL rows found. Nothing to do.")
    else:
        # Find continuous blocks for fast bulk update
        blocks = []
        block = [rows[0]]
        for r1, r2 in zip(rows, rows[1:]):
            if r2 == r1 + 1:
                block.append(r2)
            else:
                blocks.append(block)
                block = [r2]
        blocks.append(block)

        for block in blocks:
            first, last = block[0], block[-1]
            length = last - first + 1
            ws.Range(ws.Cells(first, bfg_col), ws.Cells(last, bfg_col)).Value = [["L3 - Financial Control & Tax"]] * length

        print(f"✅ Updated PRODUCT CONTROL for {len(rows)} rows in {len(blocks)} blocks.")

    # Clear filter
    ws.AutoFilterMode = False

    wb.Save()
    wb.Close(False)
    excel.Quit()

if __name__ == "__main__":
    update_product_control(r"C:\Path\To\Your\Report.xlsx")


In [ ]:
import xlwings as xw
import os
import shutil

def run_task_two():
    # === CONFIG ===
    master_file = "Base_Dashboard.xlsx"
    report_names = [
        "Investor Relations",
        "Stress Testing",
        "APEX (FC)",
        "Finance COO",
        "Financial Support"
    ]

    sheets_and_cells = [
        ("Index", "D3"),
        ("Summary", "D3"),
        ("P&L Item View", "B3"),
        ("DC by Country", "B3"),
        ("Headcount by Country", "B3"),
        ("Driller", "B3")
    ]

    with xw.App(visible=False) as app:
        for report_name in report_names:
            # 📌 Open a fresh copy each loop
            wb = app.books.open(master_file)
            
            # Get month from 'Dates'!C4
            month_text = wb.sheets['Dates'].range('C4').value  # e.g. 'May-25'
            month_part, yy_part = month_text.split('-')
            year_full = "20" + yy_part
            month_year = f"{month_part} {year_full}"  # 'May 2025'

            # Find Reporting sheet
            reporting_sheet = None
            for sh in wb.sheets:
                if 'Reporting' in sh.name:
                    reporting_sheet = sh
                    break

            if not reporting_sheet:
                raise ValueError("No sheet with 'Reporting' found.")

            # Update Reporting sheet name
            new_reporting_name = f"{month_year} Reporting"
            reporting_sheet.name = new_reporting_name

            # Update Reporting sheet cells
            reporting_ws = wb.sheets[new_reporting_name]
            reporting_ws.range("C20").value = report_name
            reporting_ws.range("B25").value = f"{month_year} Results"

            # Update other sheets/cells
            for sheet_name, cell in sheets_and_cells:
                ws = wb.sheets[sheet_name]
                ws.range(cell).value = report_name

            # Save
            output_name = f"Dashboard_{report_name}.xlsx"
            wb.save(os.path.join(os.getcwd(), output_name))
            wb.close()

        print("✅ All reports created!")

if __name__ == "__main__":
    run_task_two()


In [1]:
#Working fine for duplicating cuntries block with formula updation, but not handling blockof totals.- Task one

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.worksheet.cell_range import CellRange
from openpyxl.formula.translate import Translator
import copy


def replicate_dashboard_block_with_translated_formulas(
    file_path,
    sheet_name,
    source_start_col,
    source_end_col,
    country_list,
    header_row=3,
    text_row=5
):
    wb = load_workbook(file_path)
    ws = wb[sheet_name]

    start_col_idx = column_index_from_string(source_start_col)
    end_col_idx = column_index_from_string(source_end_col)
    num_cols_to_copy = end_col_idx - start_col_idx + 1

    # Where to start inserting the first block (+2 means 1 blank col gap)
    current_col_idx = end_col_idx + 2

    for country in country_list:
        print(f"🔹 Adding block for: {country}")

        # --- Copy the block ---
        for row in ws.iter_rows(
            min_col=start_col_idx,
            max_col=end_col_idx,
            min_row=1,
            max_row=ws.max_row
        ):
            for i, source_cell in enumerate(row):
                new_col_idx = current_col_idx + i
                new_cell = ws.cell(row=source_cell.row, column=new_col_idx)

                # ✅ Copy value or translated formula
                if source_cell.data_type == 'f':
                    orig_formula = source_cell.value
                    origin = source_cell.coordinate
                    target = new_cell.coordinate
                    new_formula = Translator(orig_formula, origin=origin).translate_formula(target)
                    new_cell.value = new_formula
                else:
                    new_cell.value = source_cell.value

                # ✅ Copy style
                new_cell.font = copy.copy(source_cell.font)
                new_cell.border = copy.copy(source_cell.border)
                new_cell.fill = copy.copy(source_cell.fill)
                new_cell.number_format = copy.copy(source_cell.number_format)
                new_cell.protection = copy.copy(source_cell.protection)
                new_cell.alignment = copy.copy(source_cell.alignment)

        # --- Handle merged header row ---
        merged_start = get_column_letter(current_col_idx)
        merged_end = get_column_letter(current_col_idx + num_cols_to_copy - 1)
        new_merge_range = f"{merged_start}{header_row}:{merged_end}{header_row}"

        # Remove overlaps if needed
        for merge in list(ws.merged_cells.ranges):
            if CellRange(new_merge_range).coord in merge.coord:
                ws.merged_cells.ranges.remove(merge)

        ws.merge_cells(new_merge_range)
        ws[f"{merged_start}{header_row}"] = country  # ✅ Only top-left cell gets value

        # Copy header style
        source_header_cell = ws[f"{source_start_col}{header_row}"]
        new_header_cell = ws[f"{merged_start}{header_row}"]
        new_header_cell.font = copy.copy(source_header_cell.font)
        new_header_cell.border = copy.copy(source_header_cell.border)
        new_header_cell.fill = copy.copy(source_header_cell.fill)
        new_header_cell.number_format = copy.copy(source_header_cell.number_format)
        new_header_cell.protection = copy.copy(source_header_cell.protection)
        new_header_cell.alignment = copy.copy(source_header_cell.alignment)

        # --- Fill ROW 4 below merged header with same country name ---
        for i in range(num_cols_to_copy):
            col_letter = get_column_letter(current_col_idx + i)
            cell = ws[f"{col_letter}{header_row + 1}"]
            cell.value = country

            # Optional: match style from the same row in the source block
            source_cell = ws[f"{get_column_letter(start_col_idx + i)}{header_row + 1}"]
            cell.font = copy.copy(source_cell.font)
            cell.border = copy.copy(source_cell.border)
            cell.fill = copy.copy(source_cell.fill)
            cell.number_format = copy.copy(source_cell.number_format)
            cell.protection = copy.copy(source_cell.protection)
            cell.alignment = copy.copy(source_cell.alignment)

        # --- Handle merged text row ---
        text_merge_start = get_column_letter(current_col_idx)
        text_merge_end = get_column_letter(current_col_idx + (column_index_from_string("Y") - start_col_idx))
        new_text_merge_range = f"{text_merge_start}{text_row}:{text_merge_end}{text_row}"

        for merge in list(ws.merged_cells.ranges):
            if CellRange(new_text_merge_range).coord in merge.coord:
                ws.merged_cells.ranges.remove(merge)

        ws.merge_cells(new_text_merge_range)
        ws[f"{text_merge_start}{text_row}"] = ws[f"{source_start_col}{text_row}"].value

        text_source_cell = ws[f"{source_start_col}{text_row}"]
        text_new_cell = ws[f"{text_merge_start}{text_row}"]
        text_new_cell.font = copy.copy(text_source_cell.font)
        text_new_cell.border = copy.copy(text_source_cell.border)
        text_new_cell.fill = copy.copy(text_source_cell.fill)
        text_new_cell.number_format = copy.copy(text_source_cell.number_format)
        text_new_cell.protection = copy.copy(text_source_cell.protection)
        text_new_cell.alignment = copy.copy(text_source_cell.alignment)

        # Advance for next block (+1 for blank col)
        current_col_idx += num_cols_to_copy + 1

    wb.save(file_path)
    wb.close()
    print("✅✅ All blocks copied successfully with translated formulas, merged headers, and country name in row 4!")


# --- EXAMPLE ---
if __name__ == "__main__":
    replicate_dashboard_block_with_translated_formulas(
        file_path="your_dashboard.xlsx",
        sheet_name="Dashboard",
        source_start_col="V",
        source_end_col="AC",
        country_list=["Paris", "Hongkong", "India"],
        header_row=3,
        text_row=5
    )


Processing... /

KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd

from openpyxl import load_workbook

# Set folder path where your Excel files are stored
folder_path = r'path_to_your_folder'  # <-- Replace this with your actual folder path

# To store flagged files and sheet names
issues = []

# Loop through each Excel file in the folder
for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file)
        try:
            wb = load_workbook(file_path, data_only=True)
            sheet_names = wb.sheetnames

            # Check only sheet 2 and 3 (index 1 and 2)
            for sheet_index in [1, 2]:
                if sheet_index < len(sheet_names):
                    ws = wb[sheet_names[sheet_index]]

                    for row in ws.iter_rows(min_row=1, values_only=True):
                        if row[2] == 'Check':  # Column C is index 2 (0-based)
                            # From Column E (index 4) onwards
                            values_to_check = row[4:]
                            if any(val != 0 and val is not None for val in values_to_check):
                                issues.append((file, sheet_names[sheet_index]))
                            break  # No need to search further in this sheet
        except Exception as e:
            print(f"Error processing {file}: {e}")

# Print the issues
if issues:
    print("❌ Files with failed validation:")
    for file_name, sheet_name in issues:
        print(f"File: {file_name}, Sheet: {sheet_name}")
else:
    print("✅ All files passed the validation checks.")


In [4]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox

# Required sheet and column mapping
required_sheets = {
    'RTN': 'RTN',
    'Location': 'Country',
    'GCB': 'GCB'
}

def select_workbook():
    """Prompts the user to select an Excel file."""
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename(
        title="Select Excel Workbook",
        filetypes=[("Excel files", "*.xlsx")]
    )
    if not file_path:
        raise Exception("No file selected.")
    return file_path

def validate_workbook(file_path):
    """Validates presence of required sheets and columns."""
    try:
        xl = pd.ExcelFile(file_path)
    except Exception as e:
        raise Exception(f"Failed to open Excel file: {e}")

    for sheet, required_col in required_sheets.items():
        if sheet not in xl.sheet_names:
            raise Exception(f"Missing required sheet: '{sheet}'")

        df = xl.parse(sheet)
        df.columns = df.columns.str.strip()  # Remove extra spaces

        if required_col not in df.columns:
            raise Exception(f"Sheet '{sheet}' must contain column '{required_col}'")
        
        if df[required_col].isnull().any():
            raise Exception(f"Column '{required_col}' in sheet '{sheet}' contains blank values")

    print("Workbook validation successful.")

if __name__ == "__main__":
    try:
        file_path = select_workbook()
        validate_workbook(file_path)
        print(f"Processing file: {file_path}")
        # Continue with your processing logic here...
    except Exception as e:
        print(f"Error: {e}")
